# Import libraries

In [1]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
import jupyros as jr
import rospy
from std_msgs.msg import String
from jupyros import ros3d
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from tf import transformations

from move_base_msgs.msg import MoveBaseActionGoal
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from actionlib_msgs.msg import GoalID, GoalStatusArray
from nav_msgs.msg import Odometry


from random import seed
from random import randint

from IPython.display import clear_output



# Define variables and publishers

## First Modality - Reach a given position

In [2]:
global goal_id
global succesful_goal
global failed_goal
global total_goal
global goal_running

goal_running = False

succesful_goal = 0
failed_goal = 0
total_goal=0

PubGoal = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 1)
PubCancel = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)

## Second & Third Modality - Drive the robot (with or without collision avoidance)

In [3]:
global linear
global angular
global linear_velocity
global angular_velocity
global velocity
global th
global obstacle_avoidance

linear = 0
angular = 0
linear_velocity = 0.2
angular_velocity = 0.5
velocity = Twist()
th = 1
obstacle_avoidance = False

pub_vel = rospy.Publisher("/cmd_vel", Twist, queue_size = 1)


# Widget Definition

## Initial choose between modalities

In [4]:
output = widgets.Output()

button1 = Button(description='Reach autonomousely a given position',
    tooltip='Reach autonomousely a given position',
layout=Layout(width='33%', height='100px', align="center", grid_area='b1'),
style=ButtonStyle(button_color='blue'))

button2 = Button(description='Drive the robot with your keyboard',
    tooltip='Drive the robot with your keyboard',
layout=Layout(width='33%',height='100px', grid_area='b2'),
style=ButtonStyle(button_color='white'))

button3 = Button(description='Drive the robot with your keyboard with assisted obstacles avoidance',
    tooltip='Drive the robot with your keyboard with assisted obstacles avoidance',
layout=Layout(width='33%', height='100px', grid_area='b3'),
style=ButtonStyle(button_color='red'))

mod_menu = HBox([button1,button2,button3])

## First Modality - reach a given position

In [5]:
choose_button = Button(description='Choose goal',
    tooltip='Choose goal',
layout=Layout(width='50%', height='100px', grid_area='b1'),
button_style='info')

cancel_button = Button(description='Cancel goal',
    tooltip='Cancel goal',
layout=Layout(width='50%',height='100px', grid_area='b2'),
button_style='danger')

yes_button = Button(description='Yes',
    tooltip='Set new goal',
layout=Layout(width='50%', height='100px', grid_area='b1'),
button_style='info')

no_button = Button(description='No',
    tooltip='Go back to the menu',
layout=Layout(width='50%',height='100px', grid_area='b2'),
button_style='danger')

goalx = widgets.FloatText(
    value=0,
    description='X:',
    disabled=False
)
goaly = widgets.FloatText(
    value=0,
    description='Y:',
    disabled=False
)

ContinueBox=HBox([yes_button,no_button])
GoalBox = HBox([goalx, goaly])


## Second & Third Modality - Drive the robot without obs avoidance & with obs avoidance

In [6]:
l_f_b = Button(description='Left and Forward',
    tooltip='Turn left and go forward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='blue'))
f_b = Button(description='Forward',
    tooltip='Go forward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='white'))
r_f_b = Button(description='Right and Forward',
    tooltip='Turn right and go forward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='red'))

l_b = Button(description='Left',
    tooltip='Turn left',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='blue'))
stop_b = Button(description='Stop',
    tooltip='Stop moving',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='white'))
r_b = Button(description='Right',
    tooltip='Turn right',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='red'))

l_b_b = Button(description='Left and Backward',
    tooltip='Turn left and go backward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='blue'))
b_b = Button(description='Backward',
    tooltip='Go backward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='white'))
r_b_b = Button(description='Right and Backward',
    tooltip='Turn right and go backward',
layout=Layout(width='50%', height='40px', grid_area='b1'),
style=ButtonStyle(button_color='red'))

declin_b = Button(description='Decrease linear vel',
    tooltip='Reduce linear velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))
resetlin_b = Button(description='reset linear velocity',
    tooltip='Reset linear velocity',
layout=Layout(width='50%', height='40px', grid_area='b1'))
incrlin_b = Button(description='Increase linear vel',
    tooltip='Increase linear velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))

decang_b = Button(description='Decrease angular vel',
    tooltip='Reduce angular velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))
resetang_b = Button(description='Reset angular velocity',
    tooltip='Reset angular velocity',
layout=Layout(width='50%', height='40px', grid_area='b1'))
incrang_b = Button(description='Increase angular vel',
    tooltip='Increase angular velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))

decall_b = Button(description='Decerease both vel',
    tooltip='Reduce angular and linear velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))
resetall_b = Button(description='reset lin & ang velocity',
    tooltip='Reset angular and linear velocity',
layout=Layout(width='50%', height='40px', grid_area='b1'))
incrall_b = Button(description='Increase lin & ang',
    tooltip='Increase angular and linear velocity by 10%',
layout=Layout(width='50%', height='40px', grid_area='b1'))

menu_button = Button(description='Back to menu',
    tooltip='Go back to the initial menu',
layout=Layout(width='100%', height='80px', grid_area='b1'),
style=ButtonStyle(button_color='orange'))


first_row=HBox([l_f_b,f_b,r_f_b])
second_row=HBox([l_b,stop_b,r_b])
third_row=HBox([l_b_b,b_b,r_b_b])

fourth_row=HBox([decang_b,resetang_b,incrang_b])
fifth_row=HBox([declin_b,resetlin_b,incrlin_b])
sixth_row=HBox([decall_b,resetall_b,incrall_b])

control = VBox([first_row,second_row,third_row,fourth_row,fifth_row,sixth_row,menu_button])
display(control)

# code for the three modalities

### First Modality - Reach a given position

In [7]:
def mod1(b):
    clear_output(wait=True)
    display(GoalBox)
    display(choose_button)
    
def send_goal_coordinates(b):
    clear_output(wait=True)
    print("Goal position: X=",goalx.value,", Y=",goaly.value)
    display(cancel_button)
    
    global goal_running
    goal_running = True
    global goal_id
    goal_id = randint(1000,10000)
    
    goal = MoveBaseActionGoal()

    goal.goal.target_pose.pose.position.x = goalx.value
    goal.goal.target_pose.pose.position.y = goaly.value
    goal.goal.target_pose.pose.orientation.w = 1
    goal.goal.target_pose.header.frame_id = "map"
    goal.goal_id.id=str(goal_id)
    
    global total_goal
    
    PubGoal.publish(goal)
    
    total_goal += 1
    
def cancel_goal(b):
    global goal_id
    global goal_running
    
    clear_output(wait=True)
    
    goal_cancel = GoalID()
    goal_cancel.id=str(goal_id)
    PubCancel.publish(goal_cancel)
    print("goal deleted")
    goal_running = False
    print("Do you want to set another goal?")
    display(ContinueBox)

def goal_status_handler(msg):
    global goal_id
    global goal_running
    global succesful_goal
    global failed_goal
    
    if goal_running == True:
        
        status=0
        if msg.status_list[0].goal_id.id == str(goal_id):
            status = msg.status_list[0].status

        if status==3:
            clear_output(wait=True)
            
            print("goal reached!")
            succesful_goal+=1
            goal_running = False
            print("Do you want to set another goal?")
            display(ContinueBox)
            
        
        if status==4:
            clear_output(wait=True)
            
            print("The goal can not be reached.")
            failed_goal+=1
            goal_running = False
            print("Do you want to set another goal?")
            display(ContinueBox)
            
def back_to_menu_mod1(b):
    clear_output(wait=True)
    display(mod_menu)



### Second & Third Modality - Drive the robot without obs avoidance & with obs avoidance

In [8]:
def mod2(b):
    clear_output(wait=True)
    display(control)
    
def mod3(b):
    global obstacle_avoidance
    obstacle_avoidance=True
    clear_output(wait=True)
    display(control)
    
def publish_vel():
    global linear
    global angular
    global linear_velocity
    global angular_velocity
    global velocity
    
    velocity.angular.z=angular_velocity*angular
    velocity.linear.x=linear_velocity*linear
    
    pub_vel.publish(velocity)    
    
    
def l_f(b):
    global linear
    global angular
    linear=1
    angular=1
    publish_vel()
    
def f(b):
    global linear
    global angular
    linear=1
    angular=0
    publish_vel()
    
def r_f(b):
    global linear
    global angular
    linear=1
    angular=-1
    publish_vel()
    
def l(b):
    global linear
    global angular
    linear=0
    angular=1
    publish_vel()
    
def stop(b):
    global linear
    global angular
    linear=0
    angular=0
    publish_vel()
    
def r(b):
    global linear
    global angular
    linear=0
    angular=-1
    publish_vel()
    
def lb(b):
    global linear
    global angular
    linear=-1
    angular=1
    publish_vel()
    
def b(b):
    global linear
    global angular
    linear=-1
    angular=0
    publish_vel()
    
def rb(b):
    global linear
    global angular
    linear=-1
    angular=-1
    publish_vel()
    
def dec_ang(b):
    global angular_velocity
    angular_velocity *= 0.9
    publish_vel()
    
def reset_ang(b):
    global angular_velocity
    angular_velocity = 1
    publish_vel()
    
def acc_ang(b):
    global angular_velocity
    angular_velocity *= 1.1
    publish_vel()
    
def dec_lin(b):
    global linear_velocity
    linear_velocity *= 0.9
    publish_vel()
    
def reset_lin(b):
    global linear_velocity
    linear_velocity = 1
    publish_vel()
    
def acc_lin(b):
    global linear_velocity
    linear_velocity *= 1.1
    publish_vel()
    
def dec_all(b):
    global linear_velocity
    global angular_velocity
    linear_velocity *= 0.9
    angular_velocity *= 0.9
    publish_vel()
    
def reset_all(b):
    global linear_velocity
    global angular_velocity
    linear_velocity = 0.5
    angular_velocity = 1
    publish_vel()
    
def acc_all(b):
    global linear_velocity
    global angular_velocity
    linear_velocity *= 1.1
    angular_velocity *= 1.1 
    publish_vel()
    
def check_walls(angle_range, min_value, max_value):
    value = 100
    
    for i in range(min_value, max_value):
        if angle_range[i] < value:
            value = angle_range[i]
    return value

def obstacle_avoidance(msg):
    
    global angular
    global linear
    global velocity
    global th
    global obstacle_avoidance
    
    if obstacle_avoidance==True: 
    
        right_wall = check_walls(msg.ranges, 0, 159)
        left_wall = check_walls(msg.ranges, 560, 719)
        front_wall = check_walls(msg.ranges, 281, 440)
    
        if front_wall < th and velocity.linear.x > 0:
                linear = 0
            
        if right_wall < th and velocity.angular.z < 0:
                angular = 0

        if left_wall < th and velocity.angular.z >0:
                angular =0
               
        publish_vel()
    
def back_to_menu_mod2_3(b):
    global linear
    global angular
    global linear_velocity
    global angular_velocity
    global velocity
    global th
    global obstacle_avoidance
    
    obstacle_avoidance = False
    linear = 0
    angular = 0
    linear_velocity = 0.2
    angular_velocity = 0.5
    
    publish_vel()
    
    clear_output(wait=True)
    display(mod_menu)
   
    
    
    

## Associate button with the right callback

### First Modality - Reach a given position

In [9]:
button1.on_click(mod1)
choose_button.on_click(send_goal_coordinates)
cancel_button.on_click(cancel_goal)
yes_button.on_click(mod1)
no_button.on_click(back_to_menu_mod1)

### Second & Third Modality - Drive the robot without obs avoidance & with obs avoidance

In [10]:
button2.on_click(mod2)
button3.on_click(mod3)

l_f_b.on_click(l_f)
f_b.on_click(f)
r_f_b.on_click(r_f)

l_b.on_click(l)
stop_b.on_click(stop)
r_b.on_click(r)

l_b_b.on_click(l_b)
b_b.on_click(b)
r_b_b.on_click(r_b)

decang_b.on_click(dec_ang)
resetang_b.on_click(reset_ang)
incrang_b.on_click(acc_ang)

declin_b.on_click(dec_lin)
resetlin_b.on_click(reset_lin)
incrlin_b.on_click(acc_lin)

decall_b.on_click(dec_all)
resetall_b.on_click(reset_all)
incrall_b.on_click(acc_all)

menu_button.on_click(back_to_menu_mod2_3)

# Functions for Data Visualisation

In [13]:
class Visualiser:
    
    def __init__(self):
        self.fig_scan= plt.figure(figsize=(6,6))
        self.ax_scan = plt.subplot(111, polar=True)
        self.ax_scan.set_thetalim(-np.pi/2,np.pi/2)
        self.ax_scan.set_theta_zero_location("N")
        self.laser  = [] 
        self.angles = []
        self.ln_scan, = self.ax_scan.plot([],[],'bo')
        
        self.fig_odom, self.ax_odom = plt.subplots()
        plt.grid(True)
        self.x_pos, self.y_pos = [], []
        self.ln_odom, = plt.plot([] ,[], 'bo')
        
        global succesful_goal
        global failed_goal
        global total_goal
        
        self.achieved = False

        self.fig_goal, self.ax_goal = plt.subplots()
        plt.grid(True)
        self.status_list = [0, 0]
        self.data = {'Successful Goals': succesful_goal, 
                     'Failed Goals': failed_goal, 
                     'Total Goals': total_goal}
        
        self.goal = list(self.data.keys())
        self.values = list(self.data.values())
        self.ln_goal = plt.bar(self.goal, self.values, color=("blue", "red", "yellow"))
        
    def plot_init(self):
    
        
        self.ax_scan.set_title("Laser Scan")
        self.ax_scan.set_ylabel("Wall distancce",fontweight="bold")
        
        self.ax_odom.set_xlim(-20, 20)
        self.ax_odom.set_ylim(-20, 20)
        self.ax_odom.set_title("Robot Position",fontweight="bold")
        self.ax_odom.set_ylabel("Y",fontweight="bold")
        self.ax_odom.set_xlabel("X",fontweight="bold")

        self.ax_goal.set_ylim(0, 10)
        
        return self.ln_scan , self.ln_odom, self.ln_goal    
        
        
    def laser_scan_callback(self, msg):
        
        self.angles = list(np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment))
        self.laser=list(msg.ranges)
        
        
    def odometry_callback(self,msg):
        self.y_pos.append(msg.pose.pose.position.y)
        self.x_pos.append(msg.pose.pose.position.x)
        
    def update_odom_plot(self, frame):
        x = self.x_pos
        y = self.y_pos
        self.ln_odom.set_data(x, y)
        return self.ln_odom
    
    
    def update_goal_plot(self, frame):
        global succesful_goal
        global failed_goal
        global total_goal
        
        
        x = total_goal
        y = failed_goal
        z = succesful_goal
        self.updated_data = {'Successful Goals': succesful_goal, 
                             'Failed Goals': failed_goal, 
                             'Total Goals': total_goal}
        
        values = list(self.updated_data.values())
        keys = list(self.updated_data.keys())
        
        self.ln_goal = plt.bar(keys, values, color=("blue", "red", "yellow"))
        
        return self.ln_goal
    
    def update_polar_plot(self, frame):  
        self.ax_scan.set_rmax(20)
        self.ln_scan.set_data(self.angles,self.laser)
        return self.ln_scan


# Init Ros and Subscribers

In [14]:

rospy.init_node("AssignmentAM")
sub_goal = rospy.Subscriber('/move_base/status', GoalStatusArray, goal_status_handler)
sub_scan = rospy.Subscriber('/scan', LaserScan, obstacle_avoidance)




# Run the code & command the Robot here

In [15]:
display(mod_menu)

# Visualize Data

In [ ]:
%matplotlib notebook

visualiser = Visualiser()
sub_odom = rospy.Subscriber('/odom', Odometry, visualiser.odometry_callback)
sub_scan2 = rospy.Subscriber('/scan', LaserScan, visualiser.laser_scan_callback)

ani_laser = FuncAnimation(visualiser.fig_scan, visualiser.update_polar_plot, blit = True)
ani_odom = FuncAnimation(visualiser.fig_odom, visualiser.update_odom_plot, init_func=visualiser.plot_init)
ani_target = FuncAnimation(visualiser.fig_goal, visualiser.update_goal_plot, init_func=visualiser.plot_init)

plt.show(block=True)